# This notebook illustrates the Gaussian Process Regression, in 2D search region.

In [1]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from jax import numpy as jnp
from jax import jit, grad,vmap
from InfoGrad import InfoGrad
from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

In [2]:
# The underlying function
def f(P):
    X = P[:,0]
    Y = P[:,1]
    return -(np.exp(-2*(X+1.5)**2 - 0.3*(Y+1.5)**2-0.3*X*Y+0.4)+np.exp(-(X-0.5)**2 - (Y-1.2)**2-0.6*X*Y)+np.exp(-0.7*(X-1.5)**2 - 0.3*(Y+1.5)**2-0.*X*Y))

# The differentiable kernel function with parameters c,l not filled.
def k(x1,x2,c,l):
    small_sig = 1e-10 # This is needed for numerical stability.
    return c * jnp.exp(-(jnp.linalg.norm(x1-x2+small_sig,axis = -1)**2) / l)

def f_sampler(X):
    Y = f(X)
    return Y+np.random.randn(*Y.shape)*0.1

def random_sample_locs(n_locs,dim_lims):
    '''
        dim_lims = [(u1,l1),(u2,l2),...,(u{space_dim},l{space_dim})]
    '''
    return (np.random.rand(n_locs, len(dim_lims)))*(dim_lims[:,-1]-dim_lims[:,0]) + dim_lims[:,0]
    

In [4]:
space_dim = 2
step_size = 0.2
n_test = 100
D  = 1
dim_lims = np.array([(-D,D),(-D,D)])
var_0  = 0.01
N_iteration = 50

xlist = np.linspace(-D, D, 100)
ylist = np.linspace(-D, D, 100)
full_X = np.meshgrid(xlist, ylist)
full_X = np.hstack([full_X[0].reshape(-1,1),full_X[1].reshape(-1,1)])

for mode in ['greedy','InfoGrad','randomWalk']:
# for mode in ['InfoGrad']:

    t = time()
# mode = 'InfoGrad'
# mode = 'randomWalk'

    X = np.array([-1,1,
                  0,0,
                  1,1,
                  1,-1,
                  -1,-1]).reshape(-1,space_dim)

    noisy_Y =  f_sampler(X)

    kernel = C(1.0, (1e-3, 1e3)) * RBF(100, (1e-6, 1e6))

    model = GPR(kernel,alpha = var_0,n_restarts_optimizer=20)

#     model = GPR(kernel,alpha = var_0,n_restarts_optimizer=10)

    model.fit(X,noisy_Y)

    curr_x = np.array([0.1,0,-0.1,0,0,-0.1]).reshape(-1,space_dim)

    gradCalc = InfoGrad(k)

    n_0 = len(X)

    x_hist = []

    mu_hist = []

    std_hist = []


    data = {}

    for n in range(N_iteration):
        new_x = []

        if mode == 'InfoGrad':
#             dx=0.1
            dx = gradCalc.dIdx(model,curr_x,X.reshape(-1,space_dim)).reshape(curr_x.shape)
            
               
            new_x = curr_x + (dx/np.linalg.norm(dx)) * step_size + np.random.randn(*curr_x.shape)*0.1 # Add some stochasticity to shake away from the corners.
            
            new_x = np.array(new_x)
            new_x[new_x>D] = D
            new_x[new_x<-D] = -D

            # Collect new data
            new_y = f_sampler(new_x)

            # Model update is mandatory, as required by sklearn.GPR

            X = np.vstack([X,new_x])
            noisy_Y = np.hstack([noisy_Y,new_y])

            model.fit(X,noisy_Y)

        elif mode == 'greedy':

            for i in range(len(curr_x)):

                x_test = random_sample_locs(n_test,dim_lims)
                
                _,std = model.predict(x_test,return_std = True)

                x_dest = x_test[np.argmax(std)]

                x_next = curr_x[i] + step_size * (x_dest-curr_x[i])/np.linalg.norm(x_dest-curr_x[i])
                
                x_next[x_next>D] = D
                x_next[x_next<-D] = -D

                y_next = f_sampler(x_next.reshape(-1,space_dim)).flatten()

                new_x.append(x_next)

                # Model update is mandatory, as required by sklearn.GPR

                X = np.vstack([X,x_next])
                noisy_Y = np.hstack([noisy_Y,y_next])

                model.fit(X,noisy_Y)
        elif mode == 'randomWalk':
#             new_x = random_sample_locs(1,np.hstack([curr_x+step_size,curr_x-step_size]).T).reshape(curr_x.shape)
            new_x = random_sample_locs(1,np.hstack([curr_x+step_size,curr_x-step_size]).reshape(-1,space_dim)).reshape(curr_x.shape)
            new_x = np.array(new_x)
            new_x[new_x>D] = D
            new_x[new_x<-D] = -D

      
            
             # Collect new data
            new_y = f_sampler(new_x)

            # Model update is mandatory, as required by sklearn.GPR
     
            X = np.vstack([X,new_x])
            noisy_Y = np.hstack([noisy_Y,new_y])

            model.fit(X,noisy_Y)
        else:
            print('Mode {} does not exist.'.format(mode))
            break

        # Update curr_x

        curr_x = np.array(new_x)

        x_hist.append(np.array(new_x))

        mu,std= model.predict(full_X.reshape(-1,space_dim),return_std=True)
        mu = mu.flatten()

        mu_hist.append(mu)
        std_hist.append(std)

    data['mu_hist'] = mu_hist
    data['std_hist'] = std_hist
    data['X'] = X
    data['noisy_Y'] = noisy_Y
    data['n_0'] = n_0
    data['n_robot'] = len(curr_x)


    with open('{}2D.pkl'.format(mode),'wb') as file:
        pkl.dump(data,file)
        
    print("Mode:",mode,"time:",time()-t)

Mode: greedy time: 77.85627746582031
Mode: InfoGrad time: 44.338162422180176
Mode: randomWalk time: 25.565160274505615
